In [1]:
#!python --version

In [2]:
#!pip install -q -U pip
#!pip install -q numpy
#!pip install -q pandas
#!pip install -q ckiptagger
#!pip install -q tqdm
#!pip install -q tensorflow==1.14.0
#!pip install -q ipywidgets
#!pip install -q gast==0.2.2

In [3]:
from ckiptagger import WS, POS
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

C:\Users\User\anaconda3\envs\AI_100\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\anaconda3\envs\AI_100\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\anaconda3\envs\AI_100\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\anaconda3\envs\AI_100\lib\site-packages\tensorflow\python\f

In [4]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [5]:
df_train

,index,class,title
0,0,體育,亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列
1,1,體育,9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜
2,2,體育,如果今年勇士奪冠，下賽季詹姆斯何去何從？
3,3,體育,超級替補！科斯塔本賽季替補出場貢獻7次助攻
4,4,體育,騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍
...,...,...,...
1795,1795,遊戲,LOL：麻辣香鍋韓服Rank不合成打野刀？電刀巨魔新套路連勝中
1796,1796,遊戲,穩住，我們能贏！因為我們擁有這種強大的力量
1797,1797,遊戲,騰訊是怎樣毀掉《鬥戰神》這款可能成神的作品的？
1798,1798,遊戲,LOL你不知道的黑科技打法！


In [6]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [7]:
train_titles.items()

dict_items([(0, '亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列'), (1, '9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜'), (2, '如果今年勇士奪冠，下賽季詹姆斯何去何從？'), (3, '超級替補！科斯塔本賽季替補出場貢獻7次助攻'), (4, '騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍'), (5, '如果朗多進入轉會市場，哪些球隊適合他？'), (6, '詹姆斯G3決殺，你怎麼看？'), (7, '大魔王帶頭唱歌！火箭這像是打季後賽？爵士神帥這話已提前投降了'), (8, '馬夏爾要去切爾西？可以商量，不過穆里尼奧的要價是4000萬加威廉'), (9, '利希施泰納宣佈賽季結束後離隊：我需要新的挑戰'), (10, '怎麼樣看待大連一方在中超聯賽第九輪取得的賽季首勝？'), (11, '科勒·卡戴珊與男友TT共進午餐，曾在他懷孕期間偷腥的渣男被原'), (12, '作為央視體育體育頻道，CCTV5一到週末就直播馬拉松你怎麼看？'), (13, '如果2018騎士奪冠，詹姆斯這個冠軍的含金量有多大？'), (14, '昔日中超金靴半場獨造6球虐爆遼足 華夏送走他後悔嗎？'), (15, 'NBA歷史排名前十都有誰？'), (16, '你希望利物浦贏得歐冠嗎？巴薩主帥巴爾韋德的回答耐人尋味'), (17, '絕殺！詹姆斯38+7再創傳奇一刻，兩護法創另類神跡更功不可沒'), (18, '再現絕殺！今天的老詹怎麼吹'), (19, '拜仁3比1逆轉科隆，J羅現世界級做餅'), (20, '白送一個點球！恆大都不要？卡納瓦羅笑容真有意思'), (21, '劉詩雯戴罪立功取勝平野美宇為中國女乒拿到世乒賽冠軍，怎麼看待這場比賽？'), (22, '天堂與地獄是有多近？看看比賽最後8秒的猛龍就知道了'), (23, '魯能隊員的身高'), (24, 'WKG&M-1世界綜合格鬥賽明星見面會引爆鵬城'), (25, '如何看待第一詹黑皮爾斯，公開贊美詹姆斯？'), (26, '粵媒：缺少外援的恆大還是不行'), (27, '熱刺作死的節奏！連墊底隊都輸，歐冠席位懸了？'), (28, '德帥談沒球隊面試布茲德里克：出於自私的原因，他在這讓我開心'), (29, '雙刃劍！山東魯能高中鋒讓人又愛又恨'), (30, '克洛普：希望凱塔能

In [8]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [9]:
ws = WS('../data/')
pos = POS('../data/')

In [10]:
train_title_cuts =[]
word_s_train = ws(
    list(train_titles.values()),
    sentence_segmentation=True,
    segment_delimiter_set = {",", "。", ":", "?", "!", ";"}
    )
word_p_train = pos(word_s_train)

In [11]:
for x, y in zip(word_s_train, word_p_train):
    train_title_cuts.append(list(zip(x, y)))

In [12]:
test_title_cuts =[]
word_s_test = ws(
    list(test_titles.values()),
    sentence_segmentation=True,
    segment_delimiter_set = {",", "。", ":", "?", "!", ";"}
    )
word_p_test = pos(word_s_test)

for x, y in zip(word_s_test, word_p_test):
    test_title_cuts.append(list(zip(x, y)))

In [13]:
train_title_cuts[120]

[('國腳', 'Na'),
 ('張呈棟', 'Nb'),
 ('：', 'COLONCATEGORY'),
 ('從', 'D'),
 ('沒', 'D'),
 ('想', 'VE'),
 ('過', 'Di'),
 ('自己', 'Nh'),
 ('會', 'D'),
 ('出', 'VC'),
 ('一', 'Neu'),
 ('本', 'Nf'),
 ('書', 'Na')]

In [14]:
words_train = []
words_test = []
words = []
for key in word_s_train:
    words_train = words_train + key
    
for key in word_s_test:
    words_test = words_test + key
    
words = list(set(words_train + words_test))

# Bag of Words (BOW)

In [15]:
word2index = {}
index2word = {}
# 產生字與index對應的關係
word2index = dict(zip(words,range(len(words))))
index2word = dict(zip(range(len(words)),words))

In [16]:
word2index['溫暖']

2366

In [17]:
index2word[1541]

'雄偉'

In [18]:
train_title_cuts[120]

[('國腳', 'Na'),
 ('張呈棟', 'Nb'),
 ('：', 'COLONCATEGORY'),
 ('從', 'D'),
 ('沒', 'D'),
 ('想', 'VE'),
 ('過', 'Di'),
 ('自己', 'Nh'),
 ('會', 'D'),
 ('出', 'VC'),
 ('一', 'Neu'),
 ('本', 'Nf'),
 ('書', 'Na')]

In [19]:
def get_bow_vector(pairs, word2index):
    vector = np.zeros(len(word2index.items()))
    for x,y in pairs:
        vector[word2index[x]] = 1
    return vector

In [20]:
get_bow_vector(train_title_cuts[120], word2index)

array([0., 0., 0., ..., 0., 0., 0.])

# 排除較無意義的詞性

In [21]:
pos_analysis = {}
for pairs in train_title_cuts:
    for word, flag in pairs:
        if flag not in pos_analysis:
            pos_analysis[flag] = set()
        pos_analysis[flag].add(word)

for flag, words in pos_analysis.items():
    print(flag, ':', list(words)[:100])
    print('=======================')

Nb : ['網傳', '科爾', '淘寶', '舒斯特爾', '保羅', '迪士尼', '卡納瓦羅', '小詹皇', '吳敬璉', '維韋杜詹霍', '鄭眼', '塔特姆', '浦發', '芒格', '里坦克', '武磊', '詹皇', '宋鴻兵', '亞洲杯', '楊帆', '大美', '華為', '隆多', '樸泰夏', '瓜迪奧拉', '中超', '國四', '霍金斯', '胤禛', '杜蕾斯', '劉偉', '朱之文', '馬克', 'WWE', '郭春林', '李盈瑩', '國六', '單薇恩', '朱嘯虎', '沃爾瑪', 'Dota2', '小智', '建聯', '景泰石', '李謹詺', '綠心', 'C羅', '夢琪', '濃眉哥', '2018-5-6', '科勒·卡戴珊', '力帆', '羅森', '宇通', '賞月夜', 'P60八核讓', '魯能', '圍甲', '易', '昌圖', '劉集', '泰拳王西', '衛報', '蘇群', '樊振東', '溫氏', '小白', '張本智和', '桑德羅', '盧', '秦緒文', '刷安徒恩', '高曉松', '沙特', '威廉姆斯', '京新', '胤祥', '雪迪龍', '劉國梁', '億航', '侯森', '亨德森', '四阿哥', '昂科威', '聯發科', '劉詩雯', '安德森', '韓長賦', '西游', '張呈棟', '堯治河', '·龜茲', '杜蘭特', '塔克', '郭煒煒', '上港', '哈希', '馮天睿', '德羅巴', '伊藤美誠']
Na : ['人和', '招人煩', '浙體', '公豬', '董事長', '腦', '2018年款', '節奏榜', '大棚', '老將', '地貌', '賽程表', '紙質', '遊客', '新星', '雨水', '破解版', '利潤', '物流', '美刀', '大獎', '心葉', '任務', '規模', '小麥孕', '身價', '質量', '洋眼', '好兄弟', '紅險', '水鄉', '大招', '入圍賽', '戰隊', '天氣', '股神', '觸手', '溢價', '地膜', '實錘', '史冊', '碗', '續航', '階段', '順口溜', '關鍵戰', '潛質', '神裝', '稅利', '鋁', '布

|         Type        |     Description    |
|:-------------------:|:------------------:|
| A                   | 非謂形容詞         |
| Caa                 | 對等連接詞         |
| Cab                 | 連接詞，如：等等   |
| Cba                 | 連接詞，如：的話   |
| Cbb                 | 關聯連接詞         |
| D                   | 副詞               |
| Da                  | 數量副詞           |
| Dfa                 | 動詞前程度副詞     |
| Dfb                 | 動詞後程度副詞     |
| Di                  | 時態標記           |
| Dk                  | 句副詞             |
| DM                  | 定量式             |
| I                   | 感嘆詞             |
| Na                  | 普通名詞           |
| Nb                  | 專有名詞           |
| Nc                  | 地方詞             |
| Ncd                 | 位置詞             |
| Nd                  | 時間詞             |
| Nep                 | 指代定詞           |
| Neqa                | 數量定詞           |
| Neqb                | 後置數量定詞       |
| Nes                 | 特指定詞           |
| Neu                 | 數詞定詞           |
| Nf                  | 量詞               |
| Ng                  | 後置詞             |
| Nh                  | 代名詞             |
| Nv                  | 名物化動詞         |
| P                   | 介詞               |
| T                   | 語助詞             |
| VA                  | 動作不及物動詞     |
| VAC                 | 動作使動動詞       |
| VB                  | 動作類及物動詞     |
| VC                  | 動作及物動詞       |
| VCL                 | 動作接地方賓語動詞 |
| VD                  | 雙賓動詞           |
| VF                  | 動作謂賓動詞       |
| VE                  | 動作句賓動詞       |
| VG                  | 分類動詞           |
| VH                  | 狀態不及物動詞     |
| VHC                 | 狀態使動動詞       |
| VI                  | 狀態類及物動詞     |
| VJ                  | 狀態及物動詞       |
| VK                  | 狀態句賓動詞       |
| VL                  | 狀態謂賓動詞       |
| V_2                 | 有                 |
|                     |                    |
| DE                  | 的之得地           |
| SHI                 | 是                 |
| FW                  | 外文               |
|                     |                    |
| COLONCATEGORY       | 冒號               |
| COMMACATEGORY       | 逗號               |
| DASHCATEGORY        | 破折號             |
| DOTCATEGORY         | 點號               |
| ETCCATEGORY         | 刪節號             |
| EXCLAMATIONCATEGORY | 驚嘆號             |
| PARENTHESISCATEGORY | 括號               |
| PAUSECATEGORY       | 頓號               |
| PERIODCATEGORY      | 句號               |
| QUESTIONCATEGORY    | 問號               |
| SEMICOLONCATEGORY   | 分號               |
| SPCHANGECATEGORY    | 雙直線             |
| WHITESPACE          | 空白               |

In [22]:
def get_bow_vector_with_selection(pairs, word2index):
    excluded_flags = ['COLONCATEGORY', 'PAUSECATEGORY', 'WHITESPACE', 'COMMACATEGORY', 'QUESTIONCATEGORY', 'EXCLAMATIONCATEGORY',
    'FW', 'T', 'DASHCATEGORY', 'I', 'PARENTHESISCATEGORY', 'PERIODCATEGORY', 'Cab', 'SEMICOLONCATEGORY', 'ETCCATEGORY',
    'DOTCATEGORY']
    vector = np.zeros(len(word2index))
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            vector[word2index[word]] += 1
    return vector

In [23]:
get_bow_vector(train_title_cuts[100], word2index)

array([0., 0., 0., ..., 0., 0., 0.])

# Cosine Similarity

In [24]:
def cosine_similarity(bow1, bow2):
    eps: float=1e-8
    nbow1 = bow1 / (np.sqrt(np.sum(bow1**2)) + eps)
    nbow2 = bow2 / (np.sqrt(np.sum(bow2**2)) + eps)
    similarity = np.dot(nbow1,nbow2)
    return similarity

In [25]:
bow1 = get_bow_vector(train_title_cuts[100], word2index)
bow2 = get_bow_vector(train_title_cuts[130], word2index)
cosine_similarity(bow1, bow2)

0.08703882746415753

In [26]:
train_title_cuts[100]

[('山東', 'Nc'),
 ('魯能', 'Nb'),
 ('有沒有', 'D'),
 ('可能', 'D'),
 ('拿到', 'VC'),
 ('今年', 'Nd'),
 ('的', 'DE'),
 ('中', 'A'),
 ('超', 'A'),
 ('冠軍', 'Na'),
 ('？', 'QUESTIONCATEGORY')]

In [27]:
train_title_cuts[130]

[('NBA', 'Nb'),
 ('和', 'Caa'),
 ('CBA', 'FW'),
 ('差距', 'Na'),
 ('在', 'P'),
 ('哪裡', 'Ncd'),
 ('？', 'QUESTIONCATEGORY'),
 ('6', 'Neu'),
 ('張', 'Nf'),
 ('圖', 'VF'),
 ('一目瞭然', 'VH'),
 ('！', 'EXCLAMATIONCATEGORY')]

In [28]:
train_title_cuts = dict(zip(range(len(train_title_cuts)),train_title_cuts))
test_title_cuts = dict(zip(range(1800, len(test_title_cuts)+1800),test_title_cuts))

# Group mean vector

In [29]:
#all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']
group_vectors = {news_class: [] for news_class in all_news_class}
i = 0
for index, pairs in sorted(train_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)
group_mean_vector

{'體育': array([0., 0., 0., ..., 0., 0., 0.]),
 '財經': array([0., 0., 0., ..., 0., 0., 0.]),
 '科技': array([0., 0., 0., ..., 0., 0., 0.]),
 '旅遊': array([0.        , 0.        , 0.00333333, ..., 0.00333333, 0.        ,
        0.        ]),
 '農業': array([0.00333333, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]),
 '遊戲': array([0.        , 0.        , 0.        , ..., 0.        , 0.00333333,
        0.        ])}

# Group mean vector: 測試

In [30]:
classification = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(test_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val 
    classification[max_class].append(index)

In [32]:
from collections import Counter
test = []
for group, ids in classification.items(): 
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)

predict 體育 :  Counter({'體育': 66, '遊戲': 9, '旅遊': 8, '財經': 7, '農業': 6, '科技': 4})
predict 財經 :  Counter({'財經': 66, '科技': 21, '農業': 11, '體育': 9, '旅遊': 9, '遊戲': 7})
predict 科技 :  Counter({'科技': 58, '財經': 14, '農業': 12, '體育': 11, '旅遊': 7, '遊戲': 4})
predict 旅遊 :  Counter({'旅遊': 62, '農業': 11, '財經': 9, '遊戲': 7, '體育': 5, '科技': 4})
predict 農業 :  Counter({'農業': 57, '旅遊': 6, '遊戲': 4, '體育': 2, '科技': 2, '財經': 1})
predict 遊戲 :  Counter({'遊戲': 69, '科技': 11, '旅遊': 8, '體育': 7, '財經': 3, '農業': 3})
